## 행렬분해(Matrix Factorization)을 기반(잠재 요인 협업 필터링, latent factor collaborative filtering)

In [1]:
import pandas as pd
import ast

In [22]:
from sklearn.decomposition import TruncatedSVD, PCA

In [27]:
import numpy as np

In [2]:
rating_data = pd.read_csv('./archive/ratings_small.csv')
movie_data = pd.read_csv('./archive/movies_metadata.csv')

C:\Users\komac\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
rating_data.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [4]:
movie_data.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


### movie_data 정리

In [5]:
movie_data = movie_data[['id', 'genres', 'title']]
movie_data = movie_data.rename(columns={'id':'movieId'})
movie_data.head(2)

,movieId,genres,title
0,862,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",Toy Story
1,8844,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",Jumanji


In [6]:
movie_data['genres'] = movie_data['genres'].apply(ast.literal_eval)

In [7]:
movie_data['genres'] = movie_data['genres'].apply(lambda x : [d['name'] for d in x]).apply(lambda x : "|".join(x))

In [8]:
movie_data.head()

,movieId,genres,title
0,862,Animation|Comedy|Family,Toy Story
1,8844,Adventure|Fantasy|Family,Jumanji
2,15602,Romance|Comedy,Grumpier Old Men
3,31357,Comedy|Drama|Romance,Waiting to Exhale
4,11862,Comedy,Father of the Bride Part II


In [9]:
print(movie_data.shape)
print(rating_data.shape)

(45466, 3)
(100004, 4)


### 데이터 정리

In [10]:
rating_data.drop('timestamp', axis = 1, inplace = True)
rating_data.head(3)

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0


In [11]:
movie_data.drop('genres', axis = 1, inplace = True)
movie_data.head(3)

,movieId,title
0,862,Toy Story
1,8844,Jumanji
2,15602,Grumpier Old Men


In [14]:
movie_data['movieId'][3]

'31357'

In [15]:
for i in range(len(movie_data)):
    if '-' in movie_data.loc[i,'movieId']:
        print(i, movie_data.loc[i,'movieId'])
        movie_data.loc[i,'movieId'] = 0
        print(i, movie_data.loc[i,'movieId'])

19730 1997-08-20
19730 0
29503 2012-09-29
29503 0
35587 2014-01-01
35587 0


In [16]:
movie_data['movieId'] = movie_data['movieId'].apply(pd.to_numeric)

In [17]:
user_movie_data = pd.merge(rating_data, movie_data, on = 'movieId')
user_movie_data.head()

,userId,movieId,rating,title
0,1,1371,2.5,Rocky III
1,4,1371,4.0,Rocky III
2,7,1371,3.0,Rocky III
3,19,1371,4.0,Rocky III
4,21,1371,3.0,Rocky III


In [18]:
user_movie_data.shape

(44994, 4)

### pivot table 만들기

In [19]:
user_movie_rating = user_movie_data.pivot_table('rating', index='userId', columns='title').fillna(0)
user_movie_rating.shape

(671, 2794)

In [20]:
user_movie_rating.head()

title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,...And God Created Woman,00 Schneider - Jagd auf Nihil Baxter,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 Angry Men,...,Zodiac,Zombie Flesh Eaters,Zombie Holocaust,Zozo,eXistenZ,xXx,¡Three Amigos!,À nos amours,Ödipussi,Şaban Oğlu Şaban
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### 컨셉: 특정 영화와 비슷한 영화를 추천

In [21]:
movie_user_rating = user_movie_rating.values.T
movie_user_rating.shape

(2794, 671)

### SVD 실행
- SVD란?
    - 특이값 분해
    - M*N 크기의 데이터 행렬을 (MXM)X(MXN)X(NXN)으로 분해

In [24]:
SVD = TruncatedSVD(n_components=12)
matrix = SVD.fit_transform(movie_user_rating)
matrix.shape

(2794, 12)

In [25]:
matrix[0]

array([ 0.53274354,  0.14545947,  0.50194014, -0.56126865, -0.39987444,
       -0.45755845,  0.3736151 , -0.06479038,  0.43737528, -0.30927334,
        0.31506483,  0.1991189 ])

In [28]:
corr = np.corrcoef(matrix)
corr.shape

(2794, 2794)

In [30]:
movie_title = user_movie_rating.columns
movie_title_list = list(movie_title)
coffey_hands = movie_title_list.index("!Women Art Revolution")

In [31]:
corr_coffey_hands = corr[coffey_hands]

In [32]:
list(movie_title[(corr_coffey_hands >= 0.9)])[:50]

['!Women Art Revolution',
 "'Twas the Night Before Christmas",
 '5 Days of War',
 'Adventures Of A Taxi Driver',
 'Air: Eating, Sleeping, Waiting and Playing',
 'An Ideal Husband',
 'Ashes and Diamonds',
 'Austin High',
 'Bangkok Traffic Love Story',
 'Barocco',
 "Big Girls Don't Cry... They Get Even",
 'Blind',
 'Candy',
 'China Gate',
 'Contraband',
 'Day of the Reaper',
 'Devil in the Flesh',
 'Dollman vs. Demonic Toys',
 "Don't Touch the Axe",
 'Dude, Where’s My Car?',
 'Fantasía... 3',
 'Five Bloody Graves',
 'Five Times Two',
 'Flesh Gordon',
 'Gaslight',
 'Girl Shy',
 'Gladiator',
 'Hard to Be a God',
 'Hatchet II',
 "Hell's Hinges",
 'Hellgate',
 'Homeless to Harvard: The Liz Murray Story',
 'Hornblower: Loyalty',
 "I Can't Sleep",
 'In the Name of the Father',
 'Intruder',
 'Judex',
 'Julie',
 'Kurukshetra',
 'Ladybug Ladybug',
 'Last Woman on Earth',
 "Let's Get Harry",
 'Me and the Colonel',
 'Night of the Demons',
 'Now and Forever',
 'One in the Chamber',
 'Oscenità',
 'Pa